# Capacity Building
## Prerequisites
- Some basic understanding of Python variables, data types, looping, conditionals and functions will be of benefit.
- Completion of  01-basic-model.ipynb, 02-flow-types.ipynb

## Stratification introduction

So far we've looked at how to create a compartmental model, add flows, request derived outputs and use different solvers. Now we'll look into stratifying a model using the [Stratification](http://summerepi.com/api/stratification.html) class.

So far, we have modelled the effects on overall population. However, the infection dynamics vary greatly with age. For example, the infection mortality may vary across different age groups with older age groups having a higher risk of death. For immunizing infections, a larger proportion of children are susceptible to infection than adults, as children have had only fewer years of exposure to infections than adults. To capture such differences that are observed in the population structure, we can use stratifications in our models. 


A commonly used stratification is age-based stratifications. Here, the basic methodology in stratification is to sub-divide the population into a number of discrete compartments classified by the age. Although age is a continuous parameter, age-structured models usually group individuals into a limited number of classes. The number of compartments will depend on factors such as data availability and the problem being addressed. In age-structured models the individuals can progress into increasingly older age classes. For example, if there are two compartments as children and adult, at a certain rate the children would move to the adult compartment and the interactions between these two compartments happen through the transmission between them. 


Such stratifications are useful in analysing childhood infections, stratifying the model to reflect different strains with different characteristics (e.g., risk of death, transmission level) and implementing age-specific interventions such as vaccine allocation and school closure. 


In this example we'll cover:

- [No stratification](#No-stratification)
- [Minimal stratification](#Minimal-stratification)
- [Population distribution](#Population-distribution)
- [Flow adjustments](#Flow-adjustments)
- [Infectiousness adjustments](#Infectiousness-adjustments)
- [Partial stratifications](#Partial-stratifications)
- [Multiple stratifications](#Multiple-stratifications)
- [Multiple interdependent stratifications](#Multiple-interdependent-stratifications)



## Data inputs
### Imports

In [1]:
# Install the summer package
# Pip is Python's standard package manager

%pip install summerepi


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Python standard library imports come first
from datetime import datetime, timedelta
from typing import List

# Then external package imports
import pandas as pd
import numpy as np
from summer import CompartmentalModel

# This time, we're going to do some interactive plotting!
pd.options.plotting.backend = "plotly"

# Define constants
GITHUB_MOH = (
    "https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/"
)

MOH_FILES = [
    "cases_malaysia",
    "deaths_malaysia",
    "hospital",
    "icu",
    "cases_state",
    "deaths_state",
]

COVID_BASE_DATE = datetime(2019, 12, 31)

region = "Malaysia"

### Utility functions

In [3]:
def fetch_mys_data(base_url: str, file_list: List[str]) -> pd.DataFrame:
    """
    Request files from MoH and combine them into one data frame.
    """
    a_list = []
    for file in file_list:
        data_type = file.split("_")[0]
        df = pd.read_csv(base_url + file + ".csv")
        df["type"] = data_type
        a_list.append(df)
    df = pd.concat(a_list)

    return df

### Get data
Now call the `fetch_mys_data` function and pass it the MoH url.<br>

In [4]:
df = fetch_mys_data(GITHUB_MOH, MOH_FILES)

# Same preprocessing steps as in notebook 1
df.loc[df["state"].isna(), "state"] = "Malaysia"
df["date"] = pd.to_datetime(df["date"])
df["date_index"] = (df["date"] - COVID_BASE_DATE).dt.days

# Configure mask for analysis
mask = (df["state"] == region) & (df["type"] == "cases")

# Also download the population data
population_url = "https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/static/population.csv"
df_pop = pd.read_csv(population_url)
initial_population = df_pop[df_pop["state"] == region]["pop"][0]

## Build a model

In [5]:
start_date = datetime(2021, 1, 1)  # Define the start date
end_date = start_date + timedelta(days=300)  # Define the duration

# Integer representation of the start and end dates.
start_date_int = (start_date - COVID_BASE_DATE).days
end_date_int = (end_date - COVID_BASE_DATE).days

Recall the `build_base_model` wrapper function from the last training session.

In [6]:
def build_base_model() -> CompartmentalModel:
    model = CompartmentalModel(
        times=(start_date_int, end_date_int),
        compartments=["S", "E", "I", "R"],
        infectious_compartments=["I"],
    )

    model.set_initial_population(
        distribution={"S": initial_population - 100, "E": 0, "I": 100}
    )
    
    return model

In [7]:
def build_model_with_flows(parameters: dict) -> CompartmentalModel:

    # This base model does not take parameters, but have a think about how it might...
    model = build_base_model()

    # Susceptible people can get infected.
    model.add_infection_frequency_flow(
        name="infection", contact_rate=parameters["contact_rate"], source="S", dest="E"
    )
    # Expose people transition to infected.
    model.add_transition_flow(
        name="progression",
        fractional_rate=parameters["progression_rate"],
        source="E",
        dest="I",
    )

    # Infectious people recover.
    model.add_transition_flow(
        name="recovery",
        fractional_rate=parameters["recovery_rate"],
        source="I",
        dest="R",
    )

    # Add an infection-specific death flow to the I compartment.
    model.add_death_flow(name="infection_death", death_rate=0.01, source="I")

    # Importantly, we will also request an output for the 'progression' flow, and name this 'notifications'
    # This will be available after a model run using the get_derived_outputs_df() method

    model.request_output_for_flow("notifications", "progression")

    return model


In [8]:
# Create a parameters dictionary - we'll reuse this whenever building the model

parameters = {
    "contact_rate": 0.5,
    "progression_rate": 1 / 3,
    "recovery_rate": 1 / 5,
}

## No stratification

With no stratification, this is just a regular SEIR model: there are 4 compartments where susceptible people get exposed, infected/infectious, some of them die, and some of them recover.

In [9]:
# Build and run model with no stratifications
model = build_model_with_flows(parameters)
model.run()

# Plot compartments
outputs_df = model.get_outputs_df()
outputs_df.plot()

In [10]:
# Create a plotting function
def plot_compartments(model: CompartmentalModel):
    outputs_df = model.get_outputs_df()
    return outputs_df.plot()

## Minimal stratification

Next, let's try a simple stratification where we split the population into 'young' (say, 0 to 18 years old) and 'old' (age 19 and above). Notice the following changes to the model outputs:

- There are now 8 compartments instead of 4: each original compartment has been split into an "old" and "young" compartment, with the original population evenly divided between them (by default).
- The model dynamics haven't changed otherwise: we will get the same results as before if we add the old and young compartments back together. This is because there is homogeneous mixing between strata and no demographic processes, etc.

In [11]:
from summer import Stratification

# Create a stratification named 'age', applying to all compartments, which
# splits the population into 'young' and 'old'.
strata = ["young", "old"]
strat = Stratification(name="age", strata=strata, compartments=["S","E", "I", "R"])

In [12]:
# Build and run model with the stratification we just defined
model = build_model_with_flows(parameters)

# After creating the compartments and flows we need to stratify the model 
# using the stratification object we created above.
model.stratify_with(strat) 

model.run()

And plot let's plot the eight epi curves ["young", "old"] * ["S","E", "I", "R"]

In [13]:
plot_compartments(model)

**Question: Why are we seeing only four curves?**

## Population distribution

We may not always wish to split the population evenly between strata. For example, we might know that 25% of the population is 'young' while 75% is 'old'. Notice that

- The stratified compartments are now split according to a 25:75 ratio into young and old respectively
- The overall model dynamics still haven't changed otherwise

In [14]:
strat = Stratification(name="age", strata=strata, compartments=["S","E", "I", "R"])

# Create our population split dictionary, whose keys match the strata
pop_split = {"young": 0.25, "old": 0.75}

# Set a population distribution
strat.set_population_split(pop_split)

# Build and run model with the stratification we just defined
model = build_model_with_flows(parameters)
model.stratify_with(strat)
model.run()


In [15]:
plot_compartments(model)

#### Reusable age stratification function

Now that we've got something meaningful, let's wrap it in a function for reuse

In [16]:
def get_age_stratification() -> Stratification:
    # Create the stratification
    strat = Stratification(name="age", strata=strata, compartments=["S","E", "I", "R"])

    # Create our population split dictionary, whose keys match the strata
    pop_split = {"young": 0.25, "old": 0.75}

    # Set a population distribution
    strat.set_population_split(pop_split)
    
    return strat

## Importation flows and stratification

We build a model with the same simple stratifaction as above; 2 age compartments, "young" and "old", but then add an importation flow to the model.  Note the following important details:

- In addition to the existing (transition) infections, there are new importatation infections for both young and old, each at half the total rate specified.  This is because split_imports is set to True, and therefore evenly divides its total amongst the target compartments.  The increase in infections compared to the previous run is consistent with this.
- The importation flow is added to the model directly, but only _after_ the Stratification has been applied.  This is because split_imports uses the model state at the time it is called in order to determine its splitting values.


In [17]:
# Build and run model with the stratification we just defined
model = build_model_with_flows(parameters)

# Stratify the model first
age_strat = get_age_stratification()
model.stratify_with(age_strat)

# Now the following call is aware of the changes made by the Stratification
model.add_importation_flow("infection_imports", 1000, "I", split_imports=True)

model.run()

In [18]:
plot_compartments(model)

## Flow adjustments

As noted so far, we've been successful in subdividing the population, but haven't actually changed our model dynamics, which is kind of boring. Next let's look at how we can adjust the flow rates based on strata. Let's assume three new facts about our disease:

- young people are twice as susceptible to infection
- old people are three times as likely to die from the infectious disease, while younger people are half as likely as under the original parameters we requested
- younger people take twice as long to recover

These inter-strata differences can be modelled using flow adjustments. Now we're seeing some genuinely new model dynamics. Note how there are fewer recovered 'old' people at the end of the model run, because of their higher death rate.

In [19]:
# Re-create the stratification object
age_strat = get_age_stratification()

# Add an adjustment to the 'infection' flow
age_strat.set_flow_adjustments(
    "infection",
    {
        "old": None,  # No adjustment for old people, use baseline requested value
        "young": 2.0,  # Young people are twice twice as susceptible to infection
    },
)

# Add an adjustment to the 'infection_death' flow
age_strat.set_flow_adjustments(
    "infection_death",
    {
        "old": 3.0,  # Older people die at three times the rate requested under the original parameters
        "young": 0.5,  # Younger people die at half the rate requested under the original parameters
    },
)

# Add an adjustment to the 'recovery' flow
age_strat.set_flow_adjustments(
    "recovery",
    {
        "old": None,  # No adjustment for old people, use baseline
        "young": 0.5,  # Young people take twice as long to recover
    },
)

# Build and run model with the stratification we just defined
model = build_model_with_flows(parameters)
model.stratify_with(age_strat)
model.run()

**Homework:**
1. Create a single data structure that represents the three disease dynamics discussed above.
2. Write a function and/or 'for loop' which calls set_flow_adjustments with each disease dynamic.

In [20]:
plot_compartments(model)

## Infectiousness adjustments

In addition to adjusting flow rates for each strata, you can also set an infectiousness level for a given strata. This affects how likely an infectious person in that stratum is to infect someone else. For example we could consider the following:

- young people are 1.2 times as infectious, because they're not wearing face masks as much
- young people are twice as susceptible to the disease, because some of them have immature immune systems


In [21]:
# Create a stratification named
age_strat = get_age_stratification()

# Add an adjustment to the 'infection' flow
age_strat.set_flow_adjustments(
    "infection",
    {
        "old": None,  # No adjustment for old people, use baseline
        "young": 2.0,  # Young people twice as susceptible
    },
)

# Add an adjustment to infectiousness levels for young people in the 'I' compartment
age_strat.add_infectiousness_adjustments(
    "I",
    {
        "old": None,  # No adjustment for old people, use baseline
        "young": 1.2,  # Young people 1.2 times more infectious
    },
)

# Build and run model with the stratification we just defined
model = build_model_with_flows(parameters)
model.stratify_with(age_strat)
model.run()

In [22]:
plot_compartments(model)

## Partial stratifications

So far we've been stratifying all compartments, but Summer allows only some of the compartments to be stratified. For example, we can stratify only the infectious compartment to model three different levels of disease severity: asymptomatic, mild and severe.

When you do a partial stratification, flow rates into that stratified compartment will automatically be adjusted with an even split to conserve the behaviour by default, e.g. a flow rate of 3 from a source will be evenly split into (1, 1, 1) across the three destinations. This behaviour can be manually overriden with a flow adjustment.

In [23]:
# This time, we'll create a function right away

def get_severity_strat() -> Stratification:
    # Create a stratification named 'severity', applying to the infectious, which
    # splits that compartment into 'asymptomatic', 'mild' and 'severe'.
    severity_strata = ["asymptomatic", "mild", "severe"]

    # Notice the new argument ["I"] for the compartment parameter.
    severity_strat = Stratification(name="severity", strata=severity_strata, compartments=["I"])

    # Set a population distribution - everyone starts out asymptomatic.
    severity_strat.set_population_split({"asymptomatic": 1.0, "mild": 0, "severe": 0})
    
    return severity_strat

# We need to call the function so we have a Stratification object to work with
severity_strat = get_severity_strat()

# Add an adjustment to the 'infection' flow, overriding default split.
severity_strat.set_flow_adjustments(
    "progression",
    {
        "asymptomatic": 0.3,  # 30% of incident cases are asymptomatic
        "mild": 0.5,  # 50% of incident cases are mild
        "severe": 0.2,  # 20% of incident cases are severe
    },
)

# Add an adjustment to the 'infection_death' flow
severity_strat.set_flow_adjustments(
    "infection_death",
    {
        "asymptomatic": 0.5,
        "mild": None,
        "severe": 1.5,
    },
)

severity_strat.add_infectiousness_adjustments(
    "I",
    {
        "asymptomatic": 0.5,
        "mild": None,
        "severe": 1.5,
    },
)

# Build and run model with the stratification we just defined
model = build_model_with_flows(parameters)
model.stratify_with(severity_strat)
model.run()

In [24]:
plot_compartments(model)

## Multiple stratifications

A model can have multiple stratifications applied in series. For example, we can add an 'age' stratification, followed by a 'severity' one.

In [25]:
### Age stratification

# Get the age stratification
age_strat = get_age_stratification()

# Add an adjustment to the 'infection' flow
age_strat.set_flow_adjustments(
    "infection",
    {
        "old": None,  # No adjustment for old people, use unstratified parameter value
        "young": 2.0,  # Young people are twice as susceptible
    },
)

# Add an adjustment to infectiousness levels for young people the 'I' compartment
age_strat.add_infectiousness_adjustments(
    "I",
    {
        "old": None,  # No adjustment for old people, use unstratified parameter value
        "young": 1.2,  # Young people are 1.2x more infectious
    },
)


### Disease severity stratification

# Get our severity stratification using the previously defined function
severity_strat = get_severity_strat()

# Add an adjustment to the 'infection' flow (overriding the default split of one third to each stratum)
severity_strat.set_flow_adjustments(
    "progression",
    {
        "asymptomatic": 0.3,  # 30% of cases are asympt.
        "mild": 0.5,  # 50% of cases are mild.
        "severe": 0.2,  # 20% of cases are severse.
    },
)

# Add an adjustment to the 'infection_death' flow
severity_strat.set_flow_adjustments(
    "infection_death",
    {
        "asymptomatic": 0.5,
        "mild": None,
        "severe": 1.5,
    },
)

severity_strat.add_infectiousness_adjustments(
    "I",
    {
        "asymptomatic": 0.5,
        "mild": None,
        "severe": 1.5,
    },
)


# Build and run model with the stratifications we just defined
model = build_model_with_flows(parameters)
# Apply age, then severity stratifications
model.stratify_with(age_strat)
model.stratify_with(severity_strat)
model.run()

In [26]:
plot_compartments(model)

## Multiple interdependent stratifications

In the previous example we assumed that the age and severity stratifications were independent. For example, we assumed that the proportion of infected people who have a disease severity of asymptomatic, mild and severe is the same for both young and old people. Perhaps, for a given disease, this is not true! it's easy to imagine an infection for which younger people tend towards being more asymptomatic, and older people tend towards having a more severe infection.

This interdependency between stratifications can be modelled using Summer, where a flow adjustment for a stratification can selectively refer to strata used for previous stratifications. You can refer to the API reference for [set_flow_adjustments](http://summerepi.com/api/stratification.html#summer.stratification.Stratification.set_flow_adjustments) for more details.

To clarify, let's consider the example described above:

In [27]:
### Age stratification

# Get the age stratification
age_strat = get_age_stratification()

### Disease severity stratification (depends on the age stratification)
# Get the severity stratification
severity_strat = get_severity_strat()

# Add an adjustment to the 'infection' flow for young people
# where younger people tend towards asymptomatic infection
young_infection_adjustments = {
    "asymptomatic": 0.5,  # 50% of cases are asympt.
    "mild": 0.4,  # 40% of cases are mild.
    "severe": 0.1,  # 10% of cases are severe.
}

severity_strat.set_flow_adjustments(
    "progression",
    young_infection_adjustments,
    source_strata={
        "age": "young"
    },  # Only apply this adjustment to flows of young people
)

# Add an adjustment to the 'infection' flow for old people
# where older people tend towards severe infection
old_infection_adjustments = {
    "asymptomatic": 0.1,  # 10% of cases are asympt.
    "mild": 0.4,  # 40% of cases are mild.
    "severe": 0.5,  # 50% of cases are severe.
}

severity_strat.set_flow_adjustments(
    "progression",
    old_infection_adjustments,
    source_strata={"age": "old"},  # Only apply this adjustment to flows of old people
)

# Add an adjustment to the 'infection_death' flow (for all age groups)
severity_strat.set_flow_adjustments(
    "infection_death",
    {
        "asymptomatic": 0.5,
        "mild": None,
        "severe": 1.5,
    },
)

# Adjust infectiousness levels (for all age groups)
severity_strat.add_infectiousness_adjustments(
    "I",
    {
        "asymptomatic": 0.5,
        "mild": None,
        "severe": 1.5,
    },
)


# Build and run model with the stratifications we just defined
model = build_model_with_flows(parameters)
# Apply age, then severity stratifications
model.stratify_with(age_strat)
model.stratify_with(severity_strat)
model.run()

In [28]:
plot_compartments(model)